## Final Forecasting Setup: Lag + Calendar Features

This notebook implements the **final and most constrained forecasting setup** in the study.

Based on insights from earlier experiments:
- Pollutant-based models  rely on contemporaneous measurements.
- AQI-only baselines  establish temporal persistence.

This notebook focuses on a **pure forecasting scenario**, using:
- Historical AQI lag features
- Calendar-based features (day of week, season), which are known at prediction time

The goal is to evaluate performance under a **realistic, leakage-safe, operational setting**.


In [114]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_absolute_error
from xgboost import XGBRegressor

In [115]:
def wrangle(filepath):
    df = pd.read_csv(filepath)
    df = df[df["city"] == "Delhi"]
    df["datetime"] = pd.to_datetime(df["datetime"])
    df = df.sort_values(by = "datetime")
    df = df.set_index("datetime")
    df.drop(columns = ["pm25",  "pm10", "no2", "so2", "co", "o3", 'latitude', 'longitude', 'temperature', 'humidity', 'wind_speed',
       'visibility', 'aqi_category', 'station', 'month'], inplace = True)
    
    return df

### Data Selection

This notebook focuses on **Delhi AQI only** to maintain a single-location time series.
Restricting the analysis to one city avoids spatial heterogeneity and allows the model
to focus purely on temporal dynamics.
Calendar features are derived solely from the datetime index and are therefore known at prediction time. Their inclusion does not introduce data leakage.


In [116]:
df = wrangle("/kaggle/input/delhi-5/delhi_ncr_aqi_dataset.csv")

In [117]:
df.head()

,date,year,day,hour,day_of_week,is_weekend,season,city,aqi
datetime,,,,,,,,,
2020-01-01 06:00:00,2020-01-01,2020,1,6,Wednesday,0,winter,Delhi,500
2020-01-01 06:00:00,2020-01-01,2020,1,6,Wednesday,0,winter,Delhi,399
2020-01-01 06:00:00,2020-01-01,2020,1,6,Wednesday,0,winter,Delhi,500
2020-01-01 06:00:00,2020-01-01,2020,1,6,Wednesday,0,winter,Delhi,500
2020-01-01 06:00:00,2020-01-01,2020,1,6,Wednesday,0,winter,Delhi,500


In [118]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 122752 entries, 2020-01-01 06:00:00 to 2025-12-31 23:00:00
Data columns (total 9 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   date         122752 non-null  object
 1   year         122752 non-null  int64 
 2   day          122752 non-null  int64 
 3   hour         122752 non-null  int64 
 4   day_of_week  122752 non-null  object
 5   is_weekend   122752 non-null  int64 
 6   season       122752 non-null  object
 7   city         122752 non-null  object
 8   aqi          122752 non-null  int64 
dtypes: int64(5), object(4)
memory usage: 9.4+ MB


### City-Level AQI Aggregation

The original dataset contains AQI measurements from multiple monitoring stations
within Delhi, resulting in multiple AQI values for the same timestamp.

To construct a single, consistent time series, AQI values are aggregated
across all stations by computing the **city-wide hourly average AQI**.

All subsequent analysis and forecasting results in this notebook
refer to this aggregated city-level AQI series.


In [119]:
df = (
    df
    .groupby(df.index)
    .agg({
        "aqi": "mean",
        "hour": "first",
        "day_of_week": "first",
        "is_weekend": "first",
        "season": "first"
    })
)


In [120]:
df.head(20)

,aqi,hour,day_of_week,is_weekend,season
datetime,,,,,
2020-01-01 06:00:00,490.857143,6,Wednesday,0,winter
2020-01-01 12:00:00,430.642857,12,Wednesday,0,winter
2020-01-01 18:00:00,477.500000,18,Wednesday,0,winter
2020-01-01 23:00:00,490.571429,23,Wednesday,0,winter
2020-01-02 06:00:00,498.785714,6,Thursday,0,winter
2020-01-02 12:00:00,431.142857,12,Thursday,0,winter
2020-01-02 18:00:00,477.714286,18,Thursday,0,winter
2020-01-02 23:00:00,495.785714,23,Thursday,0,winter
2020-01-03 06:00:00,500.000000,6,Friday,0,winter


In [121]:
ohe = OneHotEncoder(handle_unknown = "ignore", sparse_output=False)
encoded_cols = ohe.fit_transform(df[["day_of_week", "season"]])
encoded_df = pd.DataFrame(
    encoded_cols,
    columns=ohe.get_feature_names_out(["day_of_week", "season"]),
    index=df.index
)
df = pd.concat([df.drop(columns = ["day_of_week", "season"]), encoded_df], axis = 1)

In [122]:
df.head()

,aqi,hour,is_weekend,day_of_week_Friday,day_of_week_Monday,day_of_week_Saturday,day_of_week_Sunday,day_of_week_Thursday,day_of_week_Tuesday,day_of_week_Wednesday,season_monsoon,season_post_monsoon,season_summer,season_winter
datetime,,,,,,,,,,,,,,
2020-01-01 06:00:00,490.857143,6,0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
2020-01-01 12:00:00,430.642857,12,0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
2020-01-01 18:00:00,477.500000,18,0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
2020-01-01 23:00:00,490.571429,23,0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
2020-01-02 06:00:00,498.785714,6,0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0


### Lag Feature Construction

Lagged AQI values are created to model short-term temporal dependence.
Only past observations are used to ensure the forecasting setup
does not incorporate future information.


In [123]:
df = df.copy()
df = df.sort_index()

LAGS = [1, 2, 4]

for lag in LAGS:
    df[f"AQI_lag_{lag}"] = df["aqi"].shift(lag)


### Choice of Lag Intervals

Lag values are defined in terms of the underlying time resolution of the data.

- `lag1` corresponds to approximately **6 hours**
- `lag2` corresponds to approximately **12 hours**
- `lag4` corresponds to approximately **24 hours**

These lags are chosen to represent meaningful temporal horizons
(short-term persistence and daily cyclic behavior).

Intermediate lags (e.g., 18 hours) are not included in this setup
to keep the feature set compact and interpretable.


### Rolling Statistics

Rolling mean and standard deviation are computed over a short window
to capture recent trend and volatility in AQI levels.
The series is shifted by one timestep to prevent information leakage.


In [124]:
df["AQI_roll_mean_4"] = df["aqi"].shift(1).rolling(4).mean()
df["AQI_roll_std_4"]  = df["aqi"].shift(1).rolling(4).std()


In [125]:
df = df.dropna()


In [126]:
df.head()

,aqi,hour,is_weekend,day_of_week_Friday,day_of_week_Monday,day_of_week_Saturday,day_of_week_Sunday,day_of_week_Thursday,day_of_week_Tuesday,day_of_week_Wednesday,season_monsoon,season_post_monsoon,season_summer,season_winter,AQI_lag_1,AQI_lag_2,AQI_lag_4,AQI_roll_mean_4,AQI_roll_std_4
datetime,,,,,,,,,,,,,,,,,,,
2020-01-02 06:00:00,498.785714,6,0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,490.571429,477.500000,490.857143,472.392857,28.522130
2020-01-02 12:00:00,431.142857,12,0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,498.785714,490.571429,430.642857,474.375000,30.443791
2020-01-02 18:00:00,477.714286,18,0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,431.142857,498.785714,477.500000,474.500000,30.204462
2020-01-02 23:00:00,495.785714,23,0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,477.714286,431.142857,490.571429,474.553571,30.211746
2020-01-03 06:00:00,500.000000,6,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,495.785714,477.714286,498.785714,475.857143,31.228639


In [127]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 8764 entries, 2020-01-02 06:00:00 to 2025-12-31 23:00:00
Data columns (total 19 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   aqi                    8764 non-null   float64
 1   hour                   8764 non-null   int64  
 2   is_weekend             8764 non-null   int64  
 3   day_of_week_Friday     8764 non-null   float64
 4   day_of_week_Monday     8764 non-null   float64
 5   day_of_week_Saturday   8764 non-null   float64
 6   day_of_week_Sunday     8764 non-null   float64
 7   day_of_week_Thursday   8764 non-null   float64
 8   day_of_week_Tuesday    8764 non-null   float64
 9   day_of_week_Wednesday  8764 non-null   float64
 10  season_monsoon         8764 non-null   float64
 11  season_post_monsoon    8764 non-null   float64
 12  season_summer          8764 non-null   float64
 13  season_winter          8764 non-null   float64
 14  AQI_lag_1           

In [128]:
X = df.drop(columns="aqi")
y = df["aqi"]


In [129]:
split_idx = int(len(df) * 0.8)

X_train = X.iloc[:split_idx]
X_test  = X.iloc[split_idx:]

y_train = y.iloc[:split_idx]
y_test  = y.iloc[split_idx:]


### Autoregressive Model with Regularization

A Ridge regression model is used to estimate the relationship between AQI and its lagged values.
Ridge regularization helps stabilize coefficient estimates when lag features are correlated,
which is common in time-series data.


In [130]:
ridge_model = Ridge(alpha=1.0)
ridge_model.fit(X_train, y_train)

y_pred = ridge_model.predict(X_test)
print("MAE:", mean_absolute_error(y_test, y_pred))


MAE: 12.790258131387041


The reported MAE reflects out-of-sample performance on the test set,
using only historical AQI and calendar-based features.


### Ridge Coefficient Analysis

To understand which features contribute most to the forecast,
the learned Ridge regression coefficients are examined below.


In [131]:
import pandas as pd

ridge_coefs = pd.Series(
    ridge_model.coef_,
    index=X.columns
).sort_values(key=abs, ascending=False)

ridge_coefs


season_monsoon          -29.675551
season_post_monsoon      25.353661
season_winter            19.693288
season_summer           -15.371398
day_of_week_Monday       10.533981
day_of_week_Saturday     -9.536822
is_weekend               -5.706955
day_of_week_Sunday        3.829868
day_of_week_Wednesday    -2.183830
day_of_week_Friday       -1.179624
day_of_week_Tuesday      -1.049513
AQI_lag_4                 0.792841
day_of_week_Thursday     -0.414060
hour                      0.364828
AQI_roll_mean_4          -0.216516
AQI_lag_2                 0.154637
AQI_lag_1                 0.134645
AQI_roll_std_4            0.055912
dtype: float64

### Interpretation of Ridge Coefficients

The Ridge regression coefficients provide insight into the relative influence
of different feature groups on AQI predictions.

Seasonal indicators have the largest coefficients, highlighting the strong
impact of seasonal patterns on AQI levels. These coefficients are interpreted
relative to the reference season and primarily capture long-term level shifts.

Calendar effects (day of week and weekend indicators) show moderate influence,
suggesting weekly activity patterns contribute to AQI variation.

Lag-based and rolling features have smaller coefficients due to regularization
and correlation among temporal features. Their contribution is primarily in
capturing short-term dynamics rather than long-term AQI levels.


### Baseline: Naive Persistence Model

As a baseline, a naive persistence forecast is used:
the AQI at time *t* is assumed to be equal to the AQI at time *t−1*.


In [132]:
# naive forecast: AQI_t = AQI_t-1
y_naive = y_test.shift(1).dropna()
y_true  = y_test.loc[y_naive.index]

from sklearn.metrics import mean_absolute_error
print("Naive MAE:", mean_absolute_error(y_true, y_naive))


Naive MAE: 25.21877038486627


### Model Comparison

The lag-based model substantially outperforms the naive baseline,
demonstrating that the selected features capture meaningful temporal structure
beyond simple persistence.


### Walk-Forward Validation

To evaluate the model under realistic forecasting conditions, walk-forward validation
is used instead of a single static train–test split.

At each step, the model is retrained using all data available up to that point
and evaluated on the immediately following time window.


In [133]:
initial_train_size = int(len(X) * 0.6)
step_size = int(len(X) * 0.05)

maes = []

for start in range(initial_train_size, len(X), step_size):
    X_train = X.iloc[:start]
    y_train = y.iloc[:start]

    X_test = X.iloc[start:start + step_size]
    y_test = y.iloc[start:start + step_size]

    model = Ridge(alpha=1.0)
    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)
    maes.append(mean_absolute_error(y_test, y_pred))

print("Walk-forward MAE (mean):", sum(maes) / len(maes))


Walk-forward MAE (mean): 12.114437434329993


### Interpretation

The walk-forward MAE is consistent with the single split evaluation,
indicating that the model's performance is stable over time
and not driven by a favorable train–test partition.


### Non-Linear Benchmark: XGBoost

To assess whether non-linear interactions among lag and calendar features
provide additional predictive power, a gradient-boosted tree model is used
as a benchmark.


In [134]:
# Walk-forward validation parameters (same as Ridge)
initial_train_size = int(len(X) * 0.6)
step_size = int(len(X) * 0.05)

xgb_maes = []

for start in range(initial_train_size, len(X), step_size):
    X_train = X.iloc[:start]
    y_train = y.iloc[:start]

    X_test = X.iloc[start:start + step_size]
    y_test = y.iloc[start:start + step_size]

    xgb = XGBRegressor(
        n_estimators=300,
        max_depth=4,
        learning_rate=0.05,
        subsample=0.8,
        colsample_bytree=0.8,
        objective="reg:absoluteerror",
        random_state=42
    )

    xgb.fit(X_train, y_train)
    y_pred = xgb.predict(X_test)

    xgb_maes.append(mean_absolute_error(y_test, y_pred))

print("XGBoost Walk-forward MAE (mean):", sum(xgb_maes) / len(xgb_maes))


XGBoost Walk-forward MAE (mean): 8.137957723415303


### Comparison with Linear Autoregressive Model

XGBoost achieves a substantially lower MAE compared to the linear Ridge model,
indicating the presence of non-linear relationships in AQI dynamics.

However, this model trades interpretability for performance and is included
primarily as a performance benchmark.


## Final Takeaways

- AQI exhibits strong temporal dependence, which is effectively captured by lag-based features.
- Linear autoregressive models provide a robust and interpretable baseline under realistic forecasting constraints.
- Non-linear models such as XGBoost achieve lower error, indicating additional structure in the data, but at the cost of interpretability.
- Overall, lag-based forecasting remains a strong and practical approach when only past AQI and calendar information are available.
